# PREPARAÇÃO
ATENÇÂO: Lembrar de alterar o ambiente de execução para R.

In [ ]:
# Limpando Environment e Memória ===============================================
rm(list=ls(all=T))
gc()

In [ ]:
# Bibliotecas ==================================================================
# Instala e carrega pacman (facilita o gerenciamento de pacotes)
if (!requireNamespace("pacman", quietly = TRUE)) install.packages("pacman")

library(pacman)

p_load(
  tidyverse,           # Conjunto de pacotes para manipulação e visualização de dados (dplyr, ggplot2, tidyr, purrr, stringr, lubridate etc)
  plyr,                # Para usar o mapvalues
  stringi,             # Processamento de linguagem natural
  data.table,          # Manipulação eficiente e rápida de grandes tabelas de dados
  janitor,             # Limpeza e organização de dados
  openxlsx,            # Leitura e escrita de arquivos Excel (xlsx) sem necessidade do Excel instalado
  readxl,              # Leitura de arquivos Excel (xls e xlsx)
  rio,                 # Importação e exportação simplificada de vários formatos de arquivo
  vroom,               # Leitura rápida de arquivos CSV e similares, otimizando desempenho
  arrow,               # Interface para arquivos Apache Arrow e Parquet para alta performance em I/O
  googleCloudStorageR  # Interface para Google Cloud Storage, upload/download de arquivos na nuvem
  )

In [ ]:
# Acesso e autenticação ao GCS no R no Colab ===================================
# (subir arquivo da chave no ambiente antes)
chave_jason <- "/content/minha_chave_json.json"

Sys.setenv("GCS_AUTH_FILE" = chave_jason)
Sys.setenv("GAR_CLIENT_JSON" = chave_jason)

gcs_auth(json_file = chave_jason)

In [ ]:
# Ajustando diretórios =========================================================
bucket_name = ''
path_raw = ''
path_processed = ''

In [ ]:
# Abrindo Dicionários (subir arquivo do dicionário no ambiente antes) ==========
dicionarios <- lapply(
  excel_sheets("dicionários.xlsx"),
  function(sheet) {
    read_excel("dicionários.xlsx", sheet = sheet)
  }
)

dic_classes <- dicionarios[[1]]
dic_faixaeta <- dicionarios[[2]]
dic_escolaridade <- dicionarios[[3]]
dic_sexo <- dicionarios[[4]]
dic_racacor <- dicionarios[[5]]
dic_local <- dicionarios[[6]]
dic_empregador <- dicionarios[[7]]
dic_estabelecimento <- dicionarios[[8]]
dic_movimentacao <- dicionarios[[9]]
dic_trabparcial <- dicionarios[[10]]
dic_trabinterm <- dicionarios[[11]]
dic_aprendiz <- dicionarios[[12]]
dic_deficiencia <- dicionarios[[13]]
dic_tamanestabjan <- dicionarios[[14]]
dic_categoria <- dicionarios[[15]]
dic_cbo2002ocupacao <- dicionarios[[16]]

rm(dicionarios)

# PROCESSAMENTO

In [ ]:
# Funções ======================================================================
# Parâmetros -------------------------------------------------------------------
salario_minimo <- c(
  '2020' = 1045,
  '2021' = 1100,
  '2022' = 1212,
  '2023' = 1302,
  '2024' = 1412,
  '2025' = 1518
  )

# Funções auxiliares -----------------------------------------------------------
adicionar_faixaeta <- function(idade) {
  case_when(
    idade >= 10 & idade <= 14 ~ 1,
    idade >= 15 & idade <= 17 ~ 2,
    idade >= 18 & idade <= 24 ~ 3,
    idade >= 25 & idade <= 29 ~ 4,
    idade >= 30 & idade <= 39 ~ 5,
    idade >= 40 & idade <= 49 ~ 6,
    idade >= 50 & idade <= 64 ~ 7,
    idade >= 65 ~ 8,
    TRUE ~ 99
  )
}

adicionar_regiao <- function(regiao) {
  case_when(
    regiao == 1 ~ "Norte",
    regiao == 2 ~ "Nordeste",
    regiao == 3 ~ "Sudeste",
    regiao == 4 ~ "Sul",
    regiao == 5 ~ "Centro-Oeste",
    TRUE ~ "Não Informado"
  )
}

# Função PRINCIPAL de processamento --------------------------------------------
processamento <- function(dados, tipo_arquivo) {

  ## 1. Padroniza nomes --------------------------------------------------------
  setnames(
    dados,
    old = names(dados),
    new = tolower(stri_trans_general(names(dados), "Latin-ASCII"))
    )

  ## 2. Corrige sinal para CAGEDEXC ------------------------------------------
  if (tipo_arquivo == "CAGEDEXC") {
  dados[, `:=`(
    admissoes         =  as.integer(ifelse(saldomovimentacao == 1, -1, 0)),
    desligamentos     =  as.integer(ifelse(saldomovimentacao == -1, -1, 0)),
    saldomovimentacao =  as.integer(ifelse(saldomovimentacao == 1, -1, 1))
  )]
} else {
  dados[, `:=`(
    admissoes     = as.integer(saldomovimentacao == 1),
    desligamentos = as.integer(saldomovimentacao == -1)
  )]
}

  ## 4. Campos de data e salário --------------------------------------------
  dados[, ano := substr(competenciamov, 1, 4)]
  dados[, competenciamov := as.Date(
    paste0(substr(as.character(competenciamov), 1, 6), "01"),
    format = "%Y%m%d"
  )]

  dados[, salario := as.numeric(
    stringr::str_replace(stringr::str_replace_na(as.character(salario), "0"), ",", ".")
  )]
  dados[is.na(salario), salario := 0]

  dados[, salario_minimo_ano := salario_minimo[ano]]

  dados[, faixa_salarial := dplyr::case_when(
    salario <= 1 * salario_minimo_ano                         ~ "Até 1 SM",
    salario <= 2 * salario_minimo_ano                         ~ "1–2 SM",
    salario <= 3 * salario_minimo_ano                         ~ "2–3 SM",
    salario <= 4 * salario_minimo_ano                         ~ "3–4 SM",
    salario <= 5 * salario_minimo_ano                         ~ "4–5 SM",
    salario <= 10 * salario_minimo_ano                        ~ "5–10 SM",
    salario > 10 * salario_minimo_ano                         ~ "Acima de 10 SM",
    TRUE                                                      ~ "Não informado"
  )]

  ## 5. Faixa etária ---------------------------------------------------------
  dados[, faixaetaria := adicionar_faixaeta(idade)]

  ## 6. Mapeia dicionários ---------------------------------------------------
  dados[, sexo              := plyr::mapvalues(sexo,              dic_sexo$cod,          dic_sexo$nom,          warn_missing = FALSE)]
  dados[, racacor           := plyr::mapvalues(racacor,           dic_racacor$cod,       dic_racacor$nom,       warn_missing = FALSE)]
  dados[, graudeinstrucao   := plyr::mapvalues(graudeinstrucao,   dic_escolaridade$cod,  dic_escolaridade$nom,  warn_missing = FALSE)]
  dados[, faixaeta          := plyr::mapvalues(faixaetaria,       dic_faixaeta$cod,      dic_faixaeta$nom,      warn_missing = FALSE)]

  dados[, subclasse_original := subclasse]
  dados[, `:=`(
    subclasse  = plyr::mapvalues(subclasse_original, dic_classes$Subclasse, dic_classes$`Nome Subclasse`, warn_missing = FALSE),
    classe     = plyr::mapvalues(subclasse_original, dic_classes$Subclasse, dic_classes$`Nome Classe`,    warn_missing = FALSE),
    secao      = plyr::mapvalues(subclasse_original, dic_classes$Subclasse, dic_classes$`Nome Seção`,     warn_missing = FALSE)
  )]

  dados[, uf        := plyr::mapvalues(uf,        dic_local$UF,        dic_local$Nome_UF,   warn_missing = FALSE)]
  dados[, municipio := as.character(municipio)]
  dados[, municipio := plyr::mapvalues(municipio, dic_local$cod_mun_6, dic_local$nom_mun,   warn_missing = FALSE)]

  dados[, tipoempregador      := plyr::mapvalues(tipoempregador,      dic_empregador$cod,    dic_empregador$nom,    warn_missing = FALSE)]
  dados[, tipoestabelecimento := plyr::mapvalues(tipoestabelecimento, dic_estabelecimento$cod, dic_estabelecimento$nom, warn_missing = FALSE)]
  dados[, tipomovimentacao    := plyr::mapvalues(tipomovimentacao,    dic_movimentacao$cod,  dic_movimentacao$nom,  warn_missing = FALSE)]
  dados[, indtrabparcial      := plyr::mapvalues(indtrabparcial,      dic_trabparcial$cod,   dic_trabparcial$nom,   warn_missing = FALSE)]
  dados[, indtrabintermitente := plyr::mapvalues(indtrabintermitente, dic_trabinterm$cod,    dic_trabinterm$nom,    warn_missing = FALSE)]
  dados[, indicadoraprendiz   := plyr::mapvalues(indicadoraprendiz,   dic_aprendiz$cod,      dic_aprendiz$nom,      warn_missing = FALSE)]
  dados[, tipodedeficiencia   := plyr::mapvalues(tipodedeficiencia,   dic_deficiencia$cod,   dic_deficiencia$nom,   warn_missing = FALSE)]
  dados[, tamestabjan         := plyr::mapvalues(tamestabjan,         dic_tamanestabjan$cod, dic_tamanestabjan$nom, warn_missing = FALSE)]
  dados[, categoria           := plyr::mapvalues(categoria,           dic_categoria$cod,     dic_categoria$nom,     warn_missing = FALSE)]
  dados[, cbo2002ocupacao     := plyr::mapvalues(cbo2002ocupacao,     dic_cbo2002ocupacao$cod, dic_cbo2002ocupacao$nom, warn_missing = FALSE)]

  ## 7. Região nomeada -------------------------------------------------------
  dados[, regiao := adicionar_regiao(regiao)]

  ## 8. Limpa colunas auxiliares --------------------------------------------
  dados[, c("ano", "subclasse_original", "salario_minimo_ano", "faixaetaria") := NULL]

  ## 9. Retorna **linhas individuais** --------------------------------------
  return(dados)
}

In [ ]:
# Listar arquivos .txt no bucket
arquivos_txt <- gcs_list_objects(bucket = bucket_name, prefix = path_raw)$name
arquivos_txt <- arquivos_txt[grepl("\\.txt$", arquivos_txt)]

# Loop de processamento de cada arquivo bruto
for (arquivo in arquivos_txt) {

  # 1. Construir o caminho completo do arquivo no bucket para baixar
  caminho_arquivo_bruto <- arquivo

  # 2. Baixar o arquivo .txt e ler como data.table
  dados <- tryCatch({
    gcs_get_object(object_name = caminho_arquivo_bruto, bucket = bucket_name) %>%
      fread(input = ., stringsAsFactors = TRUE, encoding = "UTF-8")
  }, error = function(e) {
    cat("Erro ao ler:", arquivo, "-", e$message, "\n")
    return(NULL)
  })

  if (is.null(dados)) next

  # 3. Extrai partições do arquivo
  tipo_arquivo <- sub("(CAGED[A-Z]+).*", "\\1", basename(arquivo))
  ano_arquivo <- stringr::str_extract(arquivo, "ano=\\d{4}")
  mes_arquivo <- stringr::str_extract(arquivo, "mes=\\d{2}")

  # 4. Aplicar o processamento definido
  dados_processados <- processamento(dados, tipo_arquivo)

  # 5. Definir nome do arquivo .parquet para salvar localmente
  arquivo_parquet <- gsub(".txt", ".parquet", basename(arquivo))
  write_parquet(dados_processados, arquivo_parquet)

  # 6. Construir o caminho onde salvar no bucket processed
  caminho_arquivo_processado <- paste0(path_processed, "/", ano_arquivo, "/", mes_arquivo, "/", arquivo_parquet)

  # 7. Enviar o arquivo parquet gerado para o bucket processed no GCS
  gcs_upload(
    file = arquivo_parquet,
    bucket = bucket_name,
    name = caminho_arquivo_processado,
    upload_type = "simple",
    predefinedAcl = "bucketLevel")

  # 8. Remover arquivo parquet local para liberar espaço
  file.remove(arquivo_parquet)
   cat("Arquivo processado e enviado com sucesso:", caminho_arquivo_processado, "\n")
}

cat("\nProcessamento concluído para", length(arquivos_txt), "arquivo(s).\n")